In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np

from utils import (
    build_datasets,
    plot_grid,
    collect_paths
)

#### Build datasets
---

In [2]:
labeled_dataset = build_datasets(
    csv_path="../train",
    images_path="../avia-train/"
)

train_size = int(0.8 * len(labeled_dataset))
test_size = len(labeled_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    labeled_dataset, [train_size, test_size]
)

In [3]:
train = DataLoader(train_dataset, batch_size=100, shuffle=True)
test  = DataLoader(test_dataset, batch_size=100, shuffle=True)

#### Define the model
---

In [4]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3)
        self.max_pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(32, 16)
        self.linear_2 = nn.Linear(16, 1)
        
    def conv(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.max_pool(x)
        x = self.conv3(x)
        x = self.flatten(x)
        
        return x
        
    def ffnn(self, x):
        x = self.linear_1(x)
        x = torch.relu(x)
        x = self.linear_2(x)
        x = torch.sigmoid(x)
        
        return x
    
    def forward(self, x):
        x = self.conv(x)
        x = self.ffnn(x)
        
        return x
    
    def predict(self, x):
        with torch.no_grad():
            return self.forward(x)

In [21]:
model = CNNClassifier()
model = model.cuda()

distance =  nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [22]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

print("Trainable parameters amount: {:n}".format(params))

Trainable parameters amount: 6 577


In [23]:
torch.cuda.empty_cache()

num_epochs = 10

In [24]:
%%time
for epoch in range(num_epochs):
    loss_accumulator = []
    
    for ind, (x, y) in enumerate(train):
        x = x.cuda()
        x = x.to(torch.float32)

        y = y.cuda()
        y = y.to(torch.float32)
        y = y.unsqueeze(1)

        y_hat = model(x)
        optimizer.zero_grad()

        loss = distance(y_hat, y)
        diff = loss.item()

        loss.backward()
        optimizer.step()
        loss_accumulator.append(diff)
               
    print('epoch [{}/{}], loss: {:.5f}'.format(epoch+1, num_epochs, np.mean(loss_accumulator)))

epoch [1/10], loss: 0.44516
epoch [2/10], loss: 0.28495
epoch [3/10], loss: 0.23896
epoch [4/10], loss: 0.21587
epoch [5/10], loss: 0.20235
epoch [6/10], loss: 0.18792
epoch [7/10], loss: 0.17605
epoch [8/10], loss: 0.16583
epoch [9/10], loss: 0.14850
epoch [10/10], loss: 0.14687
Wall time: 1min 43s


#### Checks the accuracy [TODO:] and other metrics
---

In [26]:
correct = 0

for x, y in test:
    x = x.cuda()
    x = x.to(torch.float32)
    y = y.cuda()
    y = y.to(torch.float32)
    y = y.unsqueeze(1)

    y_hat = model.predict(x)
    outputs = (y_hat>0.5).to(torch.float32)
    correct += (outputs == y).float().sum()

In [27]:
print(f"Accuracy: {correct / len(test_dataset)}")

Accuracy: 0.9364542961120605


#### Plot some results [TODO]
---

In [ ]:
plot_grid(
    planes = [train_dataset[i] for i in range(9)]
)